In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from tqdm import tqdm

import pandas as pd
import time
import random

options = webdriver.ChromeOptions()
# 창 띄우지 않기
options.add_argument('--headless')

driver = webdriver.Chrome(options=options)
driver.set_window_size(width=1024, height=990)

# 큐레이터 추천 소장품 검색 페이지
base_url = f'https://www.museum.go.kr/site/main/relic/recommend/list'
# driver(chrome) open
driver.get(base_url)
# 로딩
time.sleep(2)

# 마지막 페이지 얻어오기
last_page = driver.find_element(By.CLASS_NAME, 'allPage').text.split(' ')[3]
last_page = int(last_page)
print("마지막 페이지 : ", last_page)

# 페이지별로 데이터 수집
data = []
last_page_before_fail = []
for n in tqdm(range(1, last_page+1)) :
    print(f'{n}페이지 유물 정보 수집중..')
    page_url = f'https://www.museum.go.kr/site/main/relic/recommend/list?cp={n}'
    # 탭 생성
    driver.execute_script("window.open('');")
    # 가장 끝 탭으로 이동
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(random.randint(1, 3))
    # 링크 이동
    driver.get(page_url)
    time.sleep(random.randint(2, 6))

    try :
        # 해당 페이지의 유물 정보 받아오기
        soup = bs(driver.page_source, "html.parser")
        cards = soup.find_all("li", class_="card")
    except Exception as e :
        print(f'페이지 내 <li> 받아오기 실패 --> {e}')
    
    # 진행상황 체크용
    card_cnt = 0
    for card in tqdm(cards) :
        card_cnt += 1
        print(f'{n}페이지 내 {card_cnt}번째 유물 데이터 수집 중...')
        try :
            tag_a = card.find("a", class_="img-box")
            title = card.find('div',class_='k-cura-txt').find('a').get_text()
            country_and_era = card.find('strong', string='국적/시대').find_next_sibling('p').get_text(strip=True)
            size_and_category = card.find('strong', string='크기/지정구분').find_next_sibling('p').get_text(separator='\n', strip=True)
            href = tag_a['href']
            link = f"https://www.museum.go.kr{href}"
            # print("콘텐츠 링크 : ", link)

            # 새 창 열기
            driver.execute_script("window.open('');")
            time.sleep(1)
            # 창 이동
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(random.randint(2, 5))

            # 각 유물들 상세페이지 이동 후 데이터 수집 
            driver.get(link)
            time.sleep(3)
            
            # BeautifulSoup로 html parser 이용
            soup = bs(driver.page_source, 'html.parser')

            # h5-소제목, p-본문 데이터 추출
            titles_and_paragraphs = soup.select('.prg > h5, .prg > p')
            
            # 소제목 초기화
            current_subtitle = '' 

            for item in titles_and_paragraphs : 
                # 소제목이 나오면 새로운 행에 딕셔너리 데이터 추가
                if item.name == 'h5':
                    current_subtitle = item.get_text(strip=True)
                    data.append({
                        'title' : title,
                        'era' : country_and_era,
                        'info' : size_and_category,
                        'description' : current_subtitle + '\n'
                    })
                # 본문이 나오고 저장된 소제목이 있으면 마지막 description에 본문 추가
                elif item.name == 'p' and current_subtitle:
                    data[-1]['description'] += item.get_text(strip=True)
                # 본문이 나왔으나 소제목이 없는 경우
                elif item.name == 'p' and not current_subtitle:
                    # 1500자 미만이면 마지막 decription에 덧붙이고
                    if len(data[-1]['description']) < 1500 :
                        data[-1]['description'] += item.get_text(strip=True)
                    # 1500자 이상이면 새로운 행에 딕셔너리 데이터 추가 (토큰 수 제한 피하고자)
                    else :
                        data.append({
                            'title' : title,
                            'era' : country_and_era,
                            'info' : size_and_category,
                            'description' : item.get_text(strip=True)
                        })
            driver.close()
            driver.switch_to.window(driver.window_handles[-1])
            time.sleep(2)

        except Exception as e2 :
            print(f'상세페이지 내 오류 발생 --> {e2}')
            last_page_before_fail.append(link)
   

    driver.close()
    driver.switch_to.window(driver.window_handles[-1])
    # break

# 데이터 프레임 생성 후 저장
df = pd.DataFrame(columns=['title', 'era', 'info', 'description'], data=data)
df.to_csv('./files/museum_passage.csv', encoding='utf-8-sig')
df.to_json('./files/museum_passage.json', force_ascii=False, orient='index', indent=4)
df.to_json("./files/museum_passage.jsonl", orient='records', lines=True, force_ascii=False)

if last_page_before_fail :
    fail_df = pd.DataFrame(last_page_before_fail).to_csv('./files/fail_log.csv', encoding='utf-8-sig')